In [ ]:
# This file contains the template and good practices for creating new test notebooks
# First recommendation is to write a comment with a short notebook description and important notes on the top of the first cell
# Under the comment write all necessary imports
import os
import sys
import shutil
import warnings
warnings.filterwarnings('ignore')

import numpy as np

sys.path.insert(0, '../../../seismiqb')

from seismiqb import SeismicGeometry
from seismiqb.geometry import export

In [ ]:
# The second cell contains all changeable parameters
# Overwritten values are placed between the second and the third cells, so we need to set defaults here
""" If there are some important parameters in the notebook, then you should write a docstring with them here. """
# Data creation parameters
CUBE_SHAPE = (100, 100, 100)
SEED = 42

# Preparation: workspace and fake data creation

Usually, tests start with preparation steps: we **create a workspace and fake data** for running tests on it.

It would be good if you add with a **short description of the storage structure**. It is useful in the case when a test operates with a large number of files.
Also, the description clarifies what is stored in each file.

**Storage structure:**
___



**template_test_files** (TESTS_NAME tests directory with temporary files)

&emsp;├── **test_array.npy** (Test cube data, now it is a random noise)

&emsp;├── **test_cube.sgy** (Test cube created from the `test_array.npy` data)

&emsp;└── **test_cube.meta** (`test_cube.sgy` meta file)


If the test saves any data, then it must start by creating its' **own saving directory**:  it helps to avoid mixing up files between different tests.

The next cell is placed in every test notebook that works with data dumps.

**Note**, that it is important to **use relative paths**: all directories and files created in tests are saved in a new directory which is created by the `run_notebook_test.py`.

In [ ]:
# (Re)create the test directory
OUTPUT_DIR = './template_test_files'

if os.path.exists(OUTPUT_DIR):
    shutil.rmtree(OUTPUT_DIR)

os.makedirs(OUTPUT_DIR)

**After that you can do everything you need.**

Some examples and other good practices:

In [ ]:
%%time
# Example of fake data creation
CUBE_PATH = os.path.join(OUTPUT_DIR, 'test_cube.sgy')

rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

with open(os.path.join(OUTPUT_DIR, 'test_array.npy'), 'wb') as outfile:
    np.save(outfile, data_array)

export.make_segy_from_array(array=data_array, path_segy=CUBE_PATH, zip_segy=False,
                            sample_rate=2., delay=50, pbar='t')

# Tests

The following code is a **simple test** that checks that saved and loaded data are the same.

In [ ]:
# Test example
geometry = SeismicGeometry(path=CUBE_PATH)

error_message = "Fake data creation test failed: original and loaded data are not the same"
assert np.allclose(data_array, geometry[:, :, :]), error_message

Advice for tests is to **write quite clear and detailed error messages in assertions**. In the error case all of them are printed in the cell output in the notebook and in the terminal output if you run tests from the `run_notebook_test.py` script.
These messages are helpful in finding out the reason for tests failure.


The last advice is to **split huge test notebooks** into separate notebooks for convenient tests usage.